In [ ]:
!pip install dill

In [ ]:
import sys
print (sys.version)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

In [1]:
import pickle
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from tqdm.notebook import tqdm
import dill
from copy import deepcopy

In [2]:
with open('containers.pkl','rb') as f:
    containers = pickle.load(f)
with open('id_queries.pkl','rb') as f:
    id_queries = pickle.load(f)
with open('url_docname.pkl','rb') as f:
    url_docname = pickle.load(f)
with open('docid_urls.pkl','rb') as f:
    id_urls = pickle.load(f)
with open('id_docs.pkl','rb') as f:
    id_docs = pickle.load(f)
with open('top_1000.pkl','rb') as f:
    top_1000 = pickle.load(f)
url_docname['kak-gta-san-andreas-sest-na-perednee-sidene.blogbuzz.pp.ru'] = 'content/content/20170702/doc.0087.dat'

lines =[]
with open('sample.technosphere.ir1.textrelevance.submission.txt') as f:
    for line in f:
        lines.append(line[:-1])

In [3]:
from collections import defaultdict
sample = defaultdict(list)
for line in lines[1:]:
    sample[int(line.split(',')[0])].append(int(line.split(',')[1]))

In [4]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')
rank = []
error = []
ranked = {}
for ad in tqdm(range(1,400)):
    try:
        docs = sample[ad]
        norm_tok_query = [id_queries[ad]]
        norm_corpus = []
        for doc in docs:
            norm_corpus.append(containers[url_docname[id_urls[doc]]][0])
        question_embeddings = module.signatures['question_encoder'](
        tf.constant(norm_tok_query))

        response_embeddings = module.signatures['response_encoder'](
            input=tf.constant(norm_corpus),
            context=tf.constant(norm_corpus))

        doc_scores = np.inner(question_embeddings['outputs'], response_embeddings['outputs'])
        tmp = {}
        for i in range(len(docs)):
            tmp[docs[i]] = doc_scores[0,i]
        ranked[ad] = tmp
    except:
        error.append(ad)

In [17]:
new_error = []
for ad in tqdm([75]):
    try:
        docs = sample[ad]
        norm_tok_query = [id_queries[ad]]
        norm_corpus = []
        counter = 0
        for doc in docs:
            if counter >=30:
                break
            norm_corpus.append(containers[url_docname[id_urls[doc]]][0])
            counter += 1
        question_embeddings = module.signatures['question_encoder'](
        tf.constant(norm_tok_query))

        response_embeddings = module.signatures['response_encoder'](
            input=tf.constant(norm_corpus),
            context=tf.constant(norm_corpus))

        doc_scores = np.inner(question_embeddings['outputs'], response_embeddings['outputs'])

        counter = 0
        for i in range(len(docs)):
            if counter >=30:
                break
            tmp[docs[i]] = doc_scores[0,i]
            counter += 1
        ranked[ad] = tmp
    except:
        new_error.append(ad)

In [24]:
top_10 = {}
for k in ranked.keys():
    top_10[k] = sorted(ranked[k].items(),key = lambda x: x[1])[:10]

In [25]:
ranked[399]

{38010: 0.19705153,
 38011: 0.39549857,
 38012: 0.26911062,
 38013: 0.2587829,
 38014: 0.39501983,
 38015: 0.38566628,
 38016: 0.4029767,
 38017: 0.39295006,
 38018: 0.38016173,
 38019: 0.45040858,
 38020: 0.19705153,
 38021: 0.42105237,
 38022: 0.4038689,
 38023: 0.417626,
 38024: 0.4285483,
 38025: 0.19705153,
 38026: 0.4134861,
 38027: 0.19705153,
 38028: 0.19705153,
 38029: 0.41252518,
 38030: 0.19705153,
 38031: 0.4090132,
 38032: 0.42517906,
 38033: 0.45907497,
 38034: 0.3959894,
 38035: 0.42778367,
 38036: 0.4312238,
 38037: 0.34394562,
 38038: 0.38288075,
 38039: 0.40624672,
 38040: 0.41399464,
 38041: 0.3967619,
 38042: 0.41574115,
 38043: 0.39273793,
 38044: 0.38983685,
 38045: 0.4233424,
 38046: 0.39902908,
 38047: 0.19705153,
 38048: 0.42845142,
 38049: 0.4405154,
 38050: 0.45062542,
 38051: 0.39982715,
 38052: 0.3546704,
 38053: 0.39976698,
 38054: 0.19705153,
 38055: 0.19705153,
 38056: 0.41808408,
 38057: 0.41042376,
 38058: 0.41602623,
 38059: 0.272721,
 38060: 0.197051

In [26]:
top_10[399]

[(37718, 0.13763456),
 (37717, 0.15897538),
 (7121, 0.17547387),
 (37733, 0.19225603),
 (7125, 0.19389942),
 (38113, 0.1970515),
 (38010, 0.19705153),
 (38020, 0.19705153),
 (38025, 0.19705153),
 (38027, 0.19705153)]

In [ ]:
sample[399]

In [27]:
with open('sub.txt','w') as f:
    f.write('QueryId,DocumentId\n')
    for i in range(1,400):
        for item in top_10[i]:
            f.write('{},{}\n'.format(i,item[0]))